In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import time
import random
from collections import defaultdict, deque
import heapq

# Initialize the main Tkinter window
root = tk.Tk()
root.title("Algorithm Visualizer and Comparator")

# Dynamically adjust window size
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = int(screen_width * 0.8)
window_height = int(screen_height * 0.6)
root.geometry(f"{window_width}x{window_height}")
root.configure(bg="#f0f0f0")  # Light gray background

data = []
graph = {}  # For graph algorithms

# Function to display step information in the Tkinter label
def update_step_label(step_label):
    time_label.config(text=step_label)
    root.update_idletasks()
    time.sleep(0.5)  # Slower to observe sorting

# Sorting Algorithm Implementations
def bubble_sort(data):
    for i in range(len(data) - 1):
        for j in range(len(data) - i - 1):
            if data[j] > data[j + 1]:
                data[j], data[j + 1] = data[j + 1], data[j]
            update_step_label(f"Bubble Sort - Step: Swap {data[j]} and {data[j + 1]}")
            draw_data(data, "lightblue", j, j + 1)  # Highlight swapped elements
    update_step_label("Bubble Sort Completed")

def selection_sort(data):
    for i in range(len(data)):
        min_index = i
        for j in range(i + 1, len(data)):
            if data[j] < data[min_index]:
                min_index = j
        data[i], data[min_index] = data[min_index], data[i]
        update_step_label(f"Selection Sort - Step: Swap {data[i]} with min {data[min_index]}")
        draw_data(data, "lightgreen", i, min_index)  # Highlight swapped elements
    update_step_label("Selection Sort Completed")

def insertion_sort(data):
    for i in range(1, len(data)):
        key = data[i]
        j = i - 1
        while j >= 0 and data[j] > key:
            data[j + 1] = data[j]
            j -= 1
        data[j + 1] = key
        update_step_label(f"Insertion Sort - Step: Insert {key} at position {j + 1}")
        draw_data(data, "orange", j + 1, i)  # Highlight inserted element
    update_step_label("Insertion Sort Completed")

def merge_sort(data):
    merge_sort_helper(data, 0, len(data) - 1)

def merge_sort_helper(data, left, right):
    if left < right:
        mid = (left + right) // 2
        merge_sort_helper(data, left, mid)
        merge_sort_helper(data, mid + 1, right)
        merge(data, left, mid, right)

def merge(data, left, mid, right):
    left_copy = data[left:mid + 1]
    right_copy = data[mid + 1:right + 1]
    left_cursor, right_cursor = 0, 0
    sorted_cursor = left

    while left_cursor < len(left_copy) and right_cursor < len(right_copy):
        if left_copy[left_cursor] <= right_copy[right_cursor]:
            data[sorted_cursor] = left_copy[left_cursor]
            left_cursor += 1
        else:
            data[sorted_cursor] = right_copy[right_cursor]
            right_cursor += 1
        update_step_label(f"Merging at position {sorted_cursor}")
        draw_data(data, "lightyellow", sorted_cursor, None)  # Highlight merged element
        sorted_cursor += 1

    while left_cursor < len(left_copy):
        data[sorted_cursor] = left_copy[left_cursor]
        left_cursor += 1
        sorted_cursor += 1

    while right_cursor < len(right_copy):
        data[sorted_cursor] = right_copy[right_cursor]
        right_cursor += 1
        sorted_cursor += 1

def quick_sort(data):
    quick_sort_helper(data, 0, len(data) - 1)

def quick_sort_helper(data, low, high):
    if low < high:
        pivot_index = partition(data, low, high)
        quick_sort_helper(data, low, pivot_index - 1)
        quick_sort_helper(data, pivot_index + 1, high)

def partition(data, low, high):
    pivot = data[high]
    i = low - 1
    for j in range(low, high):
        if data[j] < pivot:
            i += 1
            data[i], data[j] = data[j], data[i]
            update_step_label(f"Quick Sort - Step: Swap {data[i]} and {data[j]} with pivot {pivot}")
            draw_data(data, "pink", i, j)  # Highlight swapped elements
    data[i + 1], data[high] = data[high], data[i + 1]
    update_step_label(f"Quick Sort - Step: Move pivot {pivot} to position {i + 1}")
    draw_data(data, "lightcoral", i + 1, None)  # Highlight pivot
    return i + 1

# Graph Algorithm Implementations
class DisjointSet:
    def _init_(self, vertices):
        self.parent = {v: v for v in vertices}
        self.rank = {v: 0 for v in vertices}
    
    def find(self, item):
        if self.parent[item] != item:
            self.parent[item] = self.find(self.parent[item])
        return self.parent[item]
    
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return
        
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_y] = root_x
            self.rank[root_x] += 1

def kruskal_algorithm():
    global graph
    if not graph:
        generate_random_graph()
    
    update_step_label("Kruskal's Algorithm - Finding Minimum Spanning Tree")
    
    # Extract vertices and edges
    vertices = list(graph.keys())
    edges = []
    for u in graph:
        for v, weight in graph[u]:
            if (v, u, weight) not in edges:  # Avoid duplicates
                edges.append((u, v, weight))
    
    # Sort edges by weight
    edges.sort(key=lambda x: x[2])
    update_step_label(f"Kruskal - Sorted {len(edges)} edges by weight")
    
    # Initialize disjoint set
    ds = DisjointSet(vertices)
    
    # Result MST
    mst = []
    
    # Process edges
    for u, v, weight in edges:
        if ds.find(u) != ds.find(v):  # Check if adding this edge creates a cycle
            ds.union(u, v)
            mst.append((u, v, weight))
            update_step_label(f"Kruskal - Add edge {u}-{v} with weight {weight}")
            draw_graph(graph, highlight_edge=(u, v))
            time.sleep(0.5)
    
    # Calculate total MST weight
    total_weight = sum(w for _, _, w in mst)
    update_step_label(f"Kruskal's Algorithm Completed - MST Weight: {total_weight}")
    
    return mst

def dijkstra_algorithm():
    global graph
    if not graph:
        generate_random_graph()
    
    # Choose a random source vertex
    source = random.choice(list(graph.keys()))
    update_step_label(f"Dijkstra's Algorithm - Finding Shortest Paths from vertex {source}")
    
    # Initialize distances
    distances = {vertex: float('infinity') for vertex in graph}
    distances[source] = 0
    
    # Initialize priority queue
    priority_queue = [(0, source)]
    
    # Track visited vertices for visualization
    visited = set()
    
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)
        
        # If we've already found a better path, skip
        if current_distance > distances[current_vertex]:
            continue
        
        # Mark as visited
        visited.add(current_vertex)
        update_step_label(f"Dijkstra - Visiting vertex {current_vertex} at distance {current_distance}")
        draw_graph(graph, highlight_vertex=current_vertex, visited=visited)
        time.sleep(0.5)
        
        # Check neighbors
        for neighbor, weight in graph[current_vertex]:
            distance = current_distance + weight
            
            # If we found a better path
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
                update_step_label(f"Dijkstra - Updated distance to {neighbor}: {distance}")
    
    # Show final distances
    result_text = f"Shortest paths from vertex {source}:\n"
    for vertex, distance in distances.items():
        result_text += f"To {vertex}: {distance}\n"
    
    update_step_label("Dijkstra's Algorithm Completed")
    result_label.config(text=result_text)
    
    return distances

# Generate a random graph for visualizing graph algorithms
def generate_random_graph():
    global graph
    num_vertices = 8  # Choose a reasonable number for visualization
    vertices = list(range(1, num_vertices + 1))
    
    # Initialize empty graph
    graph = {v: [] for v in vertices}
    
    # Generate edges
    max_edges = num_vertices * (num_vertices - 1) // 2  # Maximum possible edges
    num_edges = random.randint(num_vertices, min(max_edges, num_vertices * 2))  # Reasonably connected graph
    
    edges_added = 0
    while edges_added < num_edges:
        u = random.choice(vertices)
        v = random.choice(vertices)
        
        if u != v:
            weight = random.randint(1, 10)
            
            # Check if edge already exists
            if not any(neighbor == v for neighbor, _ in graph[u]):
                graph[u].append((v, weight))
                graph[v].append((u, weight))  # For undirected graph
                edges_added += 1
    
    update_step_label(f"Generated random graph with {num_vertices} vertices and {num_edges} edges")
    draw_graph(graph)
    return graph

# Function to draw graph
def draw_graph(graph, highlight_edge=None, highlight_vertex=None, visited=None):
    canvas.delete("all")
    
    if not graph:
        return
    
    # Calculate positions for vertices
    vertices = list(graph.keys())
    center_x = window_width // 2
    center_y = window_height // 2
    radius = min(center_x, center_y) * 0.6
    
    # Calculate vertex positions in a circle
    vertex_positions = {}
    for i, v in enumerate(vertices):
        angle = 2 * 3.14159 * i / len(vertices)
        x = center_x + radius * 0.8 * float(format(round(math.cos(angle), 3))) 
        y = center_y + radius * 0.8 * float(format(round(math.sin(angle), 3)))
        vertex_positions[v] = (x, y)
    
    # Draw edges
    for u in graph:
        for v, weight in graph[u]:
            if u < v:  # Draw each edge only once
                x1, y1 = vertex_positions[u]
                x2, y2 = vertex_positions[v]
                
                # Determine edge color
                edge_color = "black"
                if highlight_edge and ((highlight_edge[0] == u and highlight_edge[1] == v) or 
                                       (highlight_edge[0] == v and highlight_edge[1] == u)):
                    edge_color = "red"
                    
                canvas.create_line(x1, y1, x2, y2, fill=edge_color, width=2)
                
                # Draw weight label
                label_x = (x1 + x2) / 2
                label_y = (y1 + y2) / 2
                canvas.create_oval(label_x-10, label_y-10, label_x+10, label_y+10, fill="white")
                canvas.create_text(label_x, label_y, text=str(weight), font=("Arial", 8))
    
    # Draw vertices
    for v, (x, y) in vertex_positions.items():
        # Determine vertex color
        vertex_color = "lightblue"
        if highlight_vertex and v == highlight_vertex:
            vertex_color = "red"
        elif visited and v in visited:
            vertex_color = "lightgreen"
            
        canvas.create_oval(x-20, y-20, x+20, y+20, fill=vertex_color, outline="black")
        canvas.create_text(x, y, text=str(v), font=("Arial", 12, "bold"))
    
    root.update_idletasks()

# Draw the current state of the array with labels
def draw_data(data, highlight_color=None, highlight_idx1=None, highlight_idx2=None):
    canvas.delete("all")
    
    # Check if we're in graph mode
    if algorithm_type.get() == "Graph Algorithm":
        if graph:
            draw_graph(graph)
        return
        
    bar_width = window_width / len(data) * 0.8
    for i, value in enumerate(data):
        # Define bar coordinates
        x0 = i * bar_width + (window_width - len(data) * bar_width) / 2
        y0 = window_height * 0.8 - value * 5  # Scale value for visual representation
        x1 = x0 + bar_width
        y1 = window_height * 0.8

        # Set default color and highlight swapped/selected elements
        color = "blue"
        if highlight_idx1 == i or highlight_idx2 == i:
            color = highlight_color if highlight_color else "red"
        
        # Draw the rectangle
        canvas.create_rectangle(x0, y0, x1, y1, fill=color, outline="black")
        
        # Draw the label above the bar
        canvas.create_text(x0 + bar_width / 2, y0 - 10, text=str(value), font=("Arial", 10), fill="black")

# Start Algorithm Based on User's Choice
def start_algorithm():
    global data, graph
    
    algorithm_choice = algorithm_type.get()
    
    if algorithm_choice == "Sorting Algorithm":
        choice = sort_choice.get()
        
        if choice == "Single Algorithm":
            algorithm = algorithm_menu.get()
            start_time = time.time()
            
            if algorithm == "Bubble Sort":
                bubble_sort(data.copy())
            elif algorithm == "Selection Sort":
                selection_sort(data.copy())
            elif algorithm == "Insertion Sort":
                insertion_sort(data.copy())
            elif algorithm == "Merge Sort":
                data_copy = data.copy()
                merge_sort(data_copy)
            elif algorithm == "Quick Sort":
                data_copy = data.copy()
                quick_sort(data_copy)

            end_time = time.time()
            time_label.config(text=f"{algorithm} Completed in {end_time - start_time:.4f} seconds")
            result_label.config(text=f"Sorted Array: {data}")

        elif choice == "All Algorithms Comparison":
            algorithms = [("Bubble Sort", bubble_sort), ("Selection Sort", selection_sort),
                        ("Insertion Sort", insertion_sort), ("Merge Sort", merge_sort),
                        ("Quick Sort", quick_sort)]
            time_text = ""

            for name, algo in algorithms:
                temp_data = data.copy()
                start_time = time.time()
                algo(temp_data)
                end_time = time.time()
                time_text += f"{name}: {end_time - start_time:.4f} seconds\n"

            time_label.config(text=time_text)
            result_label.config(text=f"Final Sorted Array: {sorted(data)}")
    
    elif algorithm_choice == "Graph Algorithm":
        algorithm = graph_algorithm_menu.get()
        start_time = time.time()
        
        if algorithm == "Kruskal's Algorithm":
            kruskal_algorithm()
        elif algorithm == "Dijkstra's Algorithm":
            dijkstra_algorithm()
            
        end_time = time.time()
        time_label.config(text=f"{algorithm} Completed in {end_time - start_time:.4f} seconds")

# Prompt for Array Input
def input_array():
    global data
    data_str = array_entry.get()
    try:
        data = list(map(int, data_str.split(',')))
        messagebox.showinfo("Input Successful", f"Array is set to: {data}")
        draw_data(data)  # Draw initial array
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a comma-separated list of integers.")

# Update UI based on algorithm type selection
def update_ui():
    algorithm_choice = algorithm_type.get()
    
    if algorithm_choice == "Sorting Algorithm":
        # Show sorting-related widgets
        array_entry.grid()
        input_button.grid()
        sort_frame.grid()
        algorithm_menu.grid()
        graph_algorithm_menu.grid_remove()
        generate_graph_button.grid_remove()
        
    elif algorithm_choice == "Graph Algorithm":
        # Show graph-related widgets
        array_entry.grid_remove()
        input_button.grid_remove()
        sort_frame.grid_remove()
        algorithm_menu.grid_remove()
        graph_algorithm_menu.grid()
        generate_graph_button.grid()
        
        # Generate a graph if needed
        if not graph:
            generate_random_graph()

# Import math module for graph visualization
import math

# Widget Layout Adjustments for Dynamic Resizing
font_size = int(window_height * 0.03)
button_font = ("Arial", int(window_height * 0.025), "bold")

# Greeting Label
greeting_label = tk.Label(root, 
                        text="Enter an array to sort or use graph algorithms below.", 
                        font=("Arial", font_size), bg="#f0f0f0")
greeting_label.grid(row=0, column=0, columnspan=3, pady=int(window_height * 0.02), sticky="ew")

# Configure the grid to allow the column to stretch
root.grid_columnconfigure(0, weight=1)
root.grid_columnconfigure(1, weight=0)
root.grid_columnconfigure(2, weight=1)

# Algorithm Type Selection
algorithm_type = tk.StringVar(value="Sorting Algorithm")
algorithm_type_frame = tk.Frame(root, bg="#f0f0f0")
algorithm_type_frame.grid(row=1, column=0, columnspan=3, pady=int(window_height * 0.01), sticky="ew")

sorting_radio = tk.Radiobutton(algorithm_type_frame, text="Sorting Algorithm", 
                              variable=algorithm_type, value="Sorting Algorithm", 
                              command=update_ui, bg="#f0f0f0")
sorting_radio.pack(side="left", padx=(0, 10))

graph_radio = tk.Radiobutton(algorithm_type_frame, text="Graph Algorithm", 
                            variable=algorithm_type, value="Graph Algorithm", 
                            command=update_ui, bg="#f0f0f0")
graph_radio.pack(side="left")

# Array Entry
array_entry = tk.Entry(root, width=int(window_width * 0.9), font=("Arial", int(window_height * 0.035))) 
array_entry.grid(row=2, column=0, padx=int(window_width * 0.01), pady=int(window_height * 0.02), sticky="ew")

# Set Array Button
input_button = tk.Button(root, text="Set Array", command=input_array, width=15, height=2, font=button_font)
input_button.grid(row=2, column=1, padx=int(window_width * 0.01))

# Sorting Options
sort_choice = tk.StringVar(value="Single Algorithm")
sort_frame = tk.Frame(root, bg="#f0f0f0")
sort_frame.grid(row=3, column=0, columnspan=3, pady=int(window_height * 0.02), sticky="ew")

single_algo_radio = tk.Radiobutton(sort_frame, text="Single Algorithm", variable=sort_choice, value="Single Algorithm", bg="#f0f0f0")
single_algo_radio.pack(side="left", padx=(0, 10))

all_algo_radio = tk.Radiobutton(sort_frame, text="All Algorithms Comparison", variable=sort_choice, value="All Algorithms Comparison", bg="#f0f0f0")
all_algo_radio.pack(side="left")

# Sorting Algorithm Menu
algorithm_menu = ttk.Combobox(root, values=["Bubble Sort", "Selection Sort", "Insertion Sort", "Merge Sort", "Quick Sort"], state="readonly")
algorithm_menu.set("Choose Sorting Algorithm")
algorithm_menu.grid(row=4, column=0, padx=int(window_width * 0.01), pady=int(window_height * 0.02), sticky="ew")

# Graph Algorithm Menu
graph_algorithm_menu = ttk.Combobox(root, values=["Kruskal's Algorithm", "Dijkstra's Algorithm"], state="readonly")
graph_algorithm_menu.set("Choose Graph Algorithm")
graph_algorithm_menu.grid(row=4, column=0, padx=int(window_width * 0.01), pady=int(window_height * 0.02), sticky="ew")
graph_algorithm_menu.grid_remove()  # Hide initially

# Generate Graph Button
generate_graph_button = tk.Button(root, text="Generate New Graph", command=generate_random_graph, width=15, height=2, font=button_font)
generate_graph_button.grid(row=4, column=1, padx=int(window_width * 0.01))
generate_graph_button.grid_remove()  # Hide initially

# Start Algorithm Button
start_button = tk.Button(root, text="Start Algorithm", command=start_algorithm, width=15, height=2, font=button_font)
start_button.grid(row=5, column=0, columnspan=3, padx=int(window_width * 0.01), pady=int(window_height * 0.02))

# Step Label
time_label = tk.Label(root, text="", font=("Arial", int(window_height * 0.025)), bg="#f0f0f0")
time_label.grid(row=6, column=0, columnspan=3, pady=int(window_height * 0.02), sticky="ew")

# Result Label
result_label = tk.Label(root, text="", font=("Arial", int(window_height * 0.025)), bg="#f0f0f0", wraplength=window_width-20)
result_label.grid(row=7, column=0, columnspan=3, pady=int(window_height * 0.02), sticky="ew")

# Create Canvas for Drawing Bars/Graphs
canvas = tk.Canvas(root, width=window_width, height=int(window_height * 0.9), bg="#ffffff")
canvas.grid(row=8, column=0, columnspan=3, pady=int(window_height * 0.02), sticky="ew")

# Start the Tkinter main loop
root.mainloop()